# 01_scrap

- scrapping data from eventseye.com and upload to DataPlatform

# Getting data from eventseye.com

In [1]:
pip install scrapy

Note: you may need to restart the kernel to use updated packages.


# Import Library

In [1]:
import scrapy
import pandas as pd
import json
import csv


from scrapy.crawler import CrawlerProcess
from scrapy.crawler import CrawlerRunner

In [3]:
class Eventseye(scrapy.Spider):
    name = "eventseye"
    start_urls = [
        'https://www.eventseye.com/fairs/st1_trade-shows_jewelry.html',
        'https://www.eventseye.com/fairs/st1_trade-shows_jewelry_1.html',
        'https://www.eventseye.com/fairs/st1_trade-shows_jewelry_2.html',
        'https://www.eventseye.com/fairs/st1_trade-shows_jewelry_3.html'
    ]
    custom_settings = {
        'FEEDS': { 'eventseye3.json': { 'format': 'json',}}
        }
        
    def parse(self, response):
        for event in response.css('tr'):
            yield {
                'Exhibition_Name': event.css('b::text').get(),
                'Describe': event.css('i::text').get(),
                'Venue': event.css('a.city::text').get(),
                'Place': event.css('a.place::text').get(),

            }

In [4]:
process = CrawlerProcess()
process.crawl(Eventseye)
process.start()

2022-10-27 12:26:30 [scrapy.utils.log] INFO: Scrapy 2.7.0 started (bot: scrapybot)
2022-10-27 12:26:30 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.8.0, Python 3.10.5 | packaged by conda-forge | (main, Jun 14 2022, 07:04:59) [GCC 10.3.0], pyOpenSSL 22.0.0 (OpenSSL 1.1.1q  5 Jul 2022), cryptography 37.0.4, Platform Linux-5.4.0-94-generic-x86_64-with-glibc2.31
2022-10-27 12:26:30 [scrapy.crawler] INFO: Overridden settings:
{}
2022-10-27 12:26:30 [py.warnings] WARNING: /opt/conda/lib/python3.10/site-packages/scrapy/utils/request.py:231: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the d

- Prepa

In [2]:
df=pd.read_json('eventseye3.json')
df

,Exhibition_Name,Describe,Venue,Place
0,None,None,None,None
1,JEWELLERY SIBERIA,"International Exhibition of Jewelry, Precious ...",Novosibirsk (Russia),Novosibirsk Expocentre
2,INHORGENTA MUNICH,"International Trade Fair for Watches, Clocks, ...",Munich (Germany),Fairground Messe München
3,THE WEDDING EXPO,South Africa’s Ultimate Wedding Industry Showcase,Pretoria (South Africa),Heartfelt Arena
4,"HKTDC HONG KONG INTERNATIONAL DIAMOND, GEM & P...","The Hong Kong International Diamond, Gem & Pea...",Hong Kong (Hong Kong),AsiaWorld - Expo
...,...,...,...,...
181,PALM BEACH JEWELRY & ANTIQUE SHOW,Jewelry & Antique Show in Palm Beach. Palm Bea...,"Palm Beach, FL (USA)",Palm Beach County Convention Center
182,REED GIFT FAIRS - SYDNEY,Biggest annual gift and Homewares event in NSW...,Sydney (Australia),ICC Sydney - International Convention Centre S...
183,THE RETAIL QUARTER SYDNEY,The Retail Quarter Sydney gathers wholesalers ...,Sydney (Australia),ICC Sydney - International Convention Centre S...
184,LUXURY BRIDAL EXPO CHICAGO OAK BROOK,Luxury Bridal Exhibition in Chicago. LUXURY BR...,"Chicago, IL (USA)",Chicago Marriott Oak Brook


In [3]:
df.isnull()

,Exhibition_Name,Describe,Venue,Place
0,True,True,True,True
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
181,False,False,False,False
182,False,False,False,False
183,False,False,False,False
184,False,False,False,False


In [4]:
df = df.dropna()
df.isnull().sum()

Exhibition_Name    0
Describe           0
Venue              0
Place              0
dtype: int64

In [13]:
def split_venue(data):
  data = data.split("(")
  return data

In [14]:
df[['Country', 'Venue_City',]] = list(map(split_venue, df['Venue']))
df['Country'] = df['Country'].astype(str)
df['Venue_City'] = df['Venue_City'].astype(str)

/tmp/ipykernel_1304/3964175750.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['Country', 'Venue_City',]] = list(map(split_venue, df['Venue']))
/tmp/ipykernel_1304/3964175750.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['Country', 'Venue_City',]] = list(map(split_venue, df['Venue']))
/tmp/ipykernel_1304/3964175750.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [15]:
df=df.drop(columns=['Venue'])

In [16]:
df

,Exhibition_Name,Describe,Place,Country,Venue_City
1,JEWELLERY SIBERIA,"International Exhibition of Jewelry, Precious ...",Novosibirsk Expocentre,Novosibirsk,Russia)
2,INHORGENTA MUNICH,"International Trade Fair for Watches, Clocks, ...",Fairground Messe München,Munich,Germany)
3,THE WEDDING EXPO,South Africa’s Ultimate Wedding Industry Showcase,Heartfelt Arena,Pretoria,South Africa)
4,"HKTDC HONG KONG INTERNATIONAL DIAMOND, GEM & P...","The Hong Kong International Diamond, Gem & Pea...",AsiaWorld - Expo,Hong Kong,Hong Kong)
5,BASELWORLD,"World Watch, Clock and Jewelry Show",Basel Fairground,Basel,Switzerland)
...,...,...,...,...,...
181,PALM BEACH JEWELRY & ANTIQUE SHOW,Jewelry & Antique Show in Palm Beach. Palm Bea...,Palm Beach County Convention Center,"Palm Beach, FL",USA)
182,REED GIFT FAIRS - SYDNEY,Biggest annual gift and Homewares event in NSW...,ICC Sydney - International Convention Centre S...,Sydney,Australia)
183,THE RETAIL QUARTER SYDNEY,The Retail Quarter Sydney gathers wholesalers ...,ICC Sydney - International Convention Centre S...,Sydney,Australia)
184,LUXURY BRIDAL EXPO CHICAGO OAK BROOK,Luxury Bridal Exhibition in Chicago. LUXURY BR...,Chicago Marriott Oak Brook,"Chicago, IL",USA)


In [17]:
def split_city(data):
  data = data.split(")")
  return data

In [21]:
df[['City','Pass']] = list(map(split_city, df['Venue_City']))
df['City'] = df['City'].astype(str)

df=df.drop(columns=['Venue_City','Pass'])

In [22]:
df

,Exhibition_Name,Describe,Place,Country,City
1,JEWELLERY SIBERIA,"International Exhibition of Jewelry, Precious ...",Novosibirsk Expocentre,Novosibirsk,Russia
2,INHORGENTA MUNICH,"International Trade Fair for Watches, Clocks, ...",Fairground Messe München,Munich,Germany
3,THE WEDDING EXPO,South Africa’s Ultimate Wedding Industry Showcase,Heartfelt Arena,Pretoria,South Africa
4,"HKTDC HONG KONG INTERNATIONAL DIAMOND, GEM & P...","The Hong Kong International Diamond, Gem & Pea...",AsiaWorld - Expo,Hong Kong,Hong Kong
5,BASELWORLD,"World Watch, Clock and Jewelry Show",Basel Fairground,Basel,Switzerland
...,...,...,...,...,...
181,PALM BEACH JEWELRY & ANTIQUE SHOW,Jewelry & Antique Show in Palm Beach. Palm Bea...,Palm Beach County Convention Center,"Palm Beach, FL",USA
182,REED GIFT FAIRS - SYDNEY,Biggest annual gift and Homewares event in NSW...,ICC Sydney - International Convention Centre S...,Sydney,Australia
183,THE RETAIL QUARTER SYDNEY,The Retail Quarter Sydney gathers wholesalers ...,ICC Sydney - International Convention Centre S...,Sydney,Australia
184,LUXURY BRIDAL EXPO CHICAGO OAK BROOK,Luxury Bridal Exhibition in Chicago. LUXURY BR...,Chicago Marriott Oak Brook,"Chicago, IL",USA


In [32]:
eventseye = df

In [33]:
print(f"Total events : {len(eventseye)}")

Total events : 179


In [35]:
eventseye = eventseye.to_dict('records')

- dump data to json file

In [36]:
with open('eventseye.json', 'w') as f:
    json.dump(eventseye, f)

# Upload data to DataPlatform

In [13]:
# import pandas as pd
# df = pd.DataFrame(columns=["Exhibition_Name", "Describe","Country","City", "Place"])

In [37]:
import requests
from tqdm.auto import tqdm

In [38]:
from dsmlibrary.datanode import DataNode

In [39]:
dir_raw_id = 62

In [40]:
datanode = DataNode()

Init DataNode sucessful!


In [41]:
datanode.upload_file(directory_id=dir_raw_id, file_path='eventseye.json')

  0%|          | 0.00/51.3k [00:00<?, ?B/s]

{'id': 274,
 'msg': 'file eventseye.json create sucessful',
 'path': 'nutnicha/DSI310-08/rawData/eventseye.json',
 'uploaded': True}